In [ ]:
import os
import time
import tqdm
import torch
import IPython
import torchvision

import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F

from torchvision import transforms
from torchvision.datasets import ImageFolder

from PIL import Image
from io import BytesIO
import glob
import random
from sklearn import svm
import pickle

# Import ResNet and change output

In [ ]:
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
model = torch.load("ResNet_Classifier_newtrainimage2.pth")

class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

model.fc = Identity()

In [ ]:
print(model._modules.keys())

odict_keys(['conv1', 'bn1', 'relu', 'maxpool', 'layer1', 'layer2', 'layer3', 'layer4', 'avgpool', 'fc'])


# Import Data

In [ ]:
device='cuda'
batch_size = 250
trans = transforms.Compose([
                            transforms.Resize([224,224]),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                 std=[0.229, 0.224, 0.225])
          ])

ML_illu_dataset = ImageFolder(r"\JupyterLab\NMA-DL_Project\ML-train_image",
                              transform=trans)
image_count = len(ML_illu_dataset)
train_indices = []
test_indices = []
for i in range(image_count):
    if random.random() < 0.1:
        test_indices.append(i)
    else:
        train_indices.append(i)

train_set = torch.utils.data.Subset(ML_illu_dataset, train_indices)
test_set = torch.utils.data.Subset(ML_illu_dataset, test_indices)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                           num_workers=6, shuffle=True, 
                                           pin_memory=False, prefetch_factor=1)

test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=6, shuffle=True, 
                                          pin_memory=False, prefetch_factor=1)

# 

In [ ]:
# Load test data
model.to(device)
model.eval()
test_images = np.empty([1,2048])
test_labels = np.empty([1])
with torch.no_grad():
    for batch in tqdm.tqdm(test_loader):
        images, labels = batch
        images = images.to(device)
        output = model(images)
        test_images = np.append(test_images, output.detach().cpu().numpy(), axis=0)
        test_labels = np.append(test_labels, labels.detach().cpu().numpy(), axis=0)

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:21<00:00,  1.92s/it]


In [ ]:
# Load train data
model.to(device)
model.eval()
train_images = np.empty([1,2048])
train_labels = np.empty([1])
with torch.no_grad():
    for batch in tqdm.tqdm(train_loader):
        images, labels = batch
        images = images.to(device)
        output = model(images)
        train_images = np.append(train_images, output.detach().cpu().numpy(), axis=0)
        train_labels = np.append(train_labels, labels.detach().cpu().numpy(), axis=0)

100%|██████████████████████████████████████████████████████████████████████████████████| 96/96 [02:18<00:00,  1.45s/it]


In [ ]:
train_labels = train_labels.astype(int)
test_labels = test_labels.astype(int)

In [ ]:
np.save("SVM_train_images.npy", train_images)
np.save("SVM_train_labels.npy", train_labels)
np.save("SVM_test_images.npy", test_images)
np.save("SVM_test_labels.npy", test_labels)

# Train the SVM

In [ ]:
x = train_images
y = train_labels
x_test = test_images
y_test = test_labels
clf = svm.LinearSVC()
clf.fit(x,y);

In [ ]:
# save the model to disk
filename = 'SVM.sav'
pickle.dump(clf, open(filename, 'wb'))
 
# # load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)

In [ ]:
acc = np.mean(np.equal(clf.predict(x_test), y_test))
print(acc)

0.9247921390778534


In [ ]:
acc = np.mean(np.equal(clf.predict(x), y))
print(acc)

0.9601944188385151
